In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd
import numpy as np
""" if(torch.cuda.is_available()):
    device = "cuda"
else:
    device = "cpu" """

device="cpu"

In [2]:
data1=pd.read_csv("./data/goodtr.csv")
data2=pd.read_csv("./data/badtr.csv")

data_train = pd.concat([data1,data2])
data_train.head(200)


,Nose X,Nose Y,Nose Z,Left Shoulder X,Left Shoulder Y,Left Shoulder Z,Right Shoulder X,Right Shoulder Y,Right Shoulder Z,Label
0,645.819778,228.747883,-495.751920,833.777542,466.400785,-179.431511,453.908920,460.894146,-190.957317,good
1,645.382538,231.751356,-479.768572,833.793488,460.541124,-157.067810,454.630623,459.233665,-191.068661,good
2,637.317886,198.295734,-426.740828,833.420715,440.161572,-157.098591,453.729401,428.858485,-178.784831,good
3,651.948700,228.353620,-363.429108,846.754837,433.019085,-107.618058,496.846695,431.808615,-130.351560,good
4,658.113556,241.229854,-425.602069,853.639832,431.371436,-108.308308,509.732590,433.925028,-124.991133,good
...,...,...,...,...,...,...,...,...,...,...
71,531.476326,420.706544,-962.552547,782.779236,473.384957,-531.126823,358.240242,511.139517,-479.938474,bad
72,776.463089,350.892634,-545.601182,799.855423,478.437424,-155.256579,471.649704,511.596479,-369.633293,bad
73,762.958908,568.746629,-1356.558838,915.561371,601.033387,-905.601482,473.728752,551.220045,-807.473488,bad
74,745.114288,567.258067,-1074.463663,925.808105,609.548478,-648.498144,473.477173,548.313046,-627.337189,bad


In [3]:

data3=pd.read_csv("./data/goodval.csv")
data4=pd.read_csv("./data/badval.csv")

data_test = pd.concat([data3,data4])
data_test.head(200)

,Nose X,Nose Y,Nose Z,Left Shoulder X,Left Shoulder Y,Left Shoulder Z,Right Shoulder X,Right Shoulder Y,Right Shoulder Z,Label
0,629.449768,287.503946,-433.363738,804.088211,473.737378,-177.699909,483.490562,482.585363,-174.759039,good
1,703.323135,287.549565,-363.598237,879.282303,479.812174,-168.614194,528.779068,483.545852,-141.752150,good
2,656.710587,202.545190,-441.115794,867.903748,429.677567,-206.222520,464.457474,429.084349,-195.042815,good
3,736.965332,278.175781,-723.139172,911.553497,470.187292,-381.313992,519.369125,462.966356,-415.945730,good
4,664.094315,245.109444,-588.598452,900.551300,461.928535,-284.289458,472.788200,459.991164,-292.915313,good
5,698.764801,268.351686,-703.565011,929.103317,479.501209,-411.444726,472.503510,483.020782,-381.039248,good
6,696.363297,284.372070,-602.703695,911.474686,500.466771,-321.058617,473.740311,507.648783,-309.163857,good
7,695.806046,272.200098,-573.550444,899.144821,498.546610,-305.249355,473.371544,504.808345,-290.026531,good
8,695.218887,267.044764,-468.708386,893.526230,498.680248,-202.003512,472.515564,502.729268,-240.985043,good
9,694.290161,269.613333,-425.918226,882.824860,493.672800,-102.939910,472.936592,495.817537,-129.907740,good


In [4]:
data = pd.concat([data_train,data_test])

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
import torch.utils
from pickle import dump
encoder = LabelEncoder()
data["Label"] = encoder.fit_transform(data["Label"])
X=data.drop(columns="Label").to_numpy(dtype=np.float32)
y=data["Label"].to_numpy(dtype=np.float32)
scaler=MinMaxScaler()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

train_dataset= torch.utils.data.TensorDataset(torch.tensor(X_train),torch.tensor(y_train))

test_dataset= torch.utils.data.TensorDataset(torch.tensor(X_test),torch.tensor(y_test))
dump(scaler, open('scaler.pkl', 'wb'))

train_dataloader=torch.utils.data.DataLoader(train_dataset,batch_size=4,num_workers=3)
test_dataloader = torch.utils.data.DataLoader(test_dataset,batch_size=1,num_workers=3)

In [6]:

class slouch_detection(nn.Module):
    def __init__(self):
        super(slouch_detection,self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(9,36),
            nn.ReLU(),
            nn.Linear(36,15),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(15,12),
            nn.ReLU(),
            nn.Linear(12,6),
            nn.ReLU(),
            nn.Linear(6,3),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(3,1),
            nn.Sigmoid()
        )
    def forward(self,x):
        return self.layers(x)


In [7]:
model = slouch_detection().to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()  # Assuming you're using Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # You can adjust the learning rate as needed


In [8]:
total=0
for params in model.parameters():
    total+=params.numel()
total

1210

In [9]:
def train_model(model, train_loader, optimizer, criterion, epochs,device):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}'):
            optimizer.zero_grad()
            labels=labels.unsqueeze(dim=1).to(device)
            outputs = model(inputs.to(device))
            #print(outputs.size(),labels.unsqueeze(dim=1).size())
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

            
            # Calculate accuracy
            predicted = torch.round(outputs)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            print()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct / total
        tqdm.write(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

# Testing loop
def test_model(model, test_loader, criterion,device,best_acc):
    model.eval()
    with torch.inference_mode():
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in tqdm(test_loader, desc='Testing'):
            labels=labels.unsqueeze(dim=1).to(device)
            outputs = model(inputs.to(device))
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)

            predicted = torch.round(outputs)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        test_loss = running_loss / len(test_loader.dataset)
        test_acc = correct / total
        if(best_acc<test_acc):
            torch.save(model.state_dict(),"slouch_detector.pt")
            best_acc=test_acc
        tqdm.write(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


In [10]:
train_model(model, train_dataloader, optimizer, criterion,device=device, epochs=100)  # Adjust epochs as needed
best_acc=0
test_model(model, test_dataloader, criterion,device,best_acc)

Epoch 1/100: 100%|██████████| 30/30 [00:00<00:00, 63.88it/s]
































Epoch 1/100, Loss: 0.7654, Accuracy: 0.4615


Epoch 2/100: 100%|██████████| 30/30 [00:00<00:00, 127.75it/s]
































Epoch 2/100, Loss: 0.7645, Accuracy: 0.4615


Epoch 3/100: 100%|██████████| 30/30 [00:00<00:00, 142.26it/s]


...

Epoch 100/100:  23%|██▎       | 7/30 [00:00<00:00, 68.54it/s]

Epoch 100/100: 100%|██████████| 30/30 [00:00<00:00, 136.00it/s]

























Epoch 100/100, Loss: 0.7206, Accuracy: 0.4615


Testing: 100%|██████████| 59/59 [00:00<00:00, 395.77it/s]

Test Loss: 0.7092, Test Accuracy: 0.4915
